# 🏡 House Price Prediction - FastAPI Project

This notebook covers:
- Generating synthetic data
- Training a Linear Regression model
- Saving the model
- Instructions to run a FastAPI app for inference

In [ ]:
# 📦 Install required libraries
%pip install scikit-learn pandas numpy joblib fastapi uvicorn --quiet

## 🔧 Generate Synthetic Data and Train Model

In [ ]:
import numpy as np # type: ignore
import pandas as pd # type: ignore
from sklearn.linear_model import LinearRegression # type: ignore
from sklearn.model_selection import train_test_split # type: ignore
import joblib # type: ignore

# Generate synthetic data
np.random.seed(42)
data_size = 1000

square_footage = np.random.randint(500, 5000, size=data_size)
bedrooms = np.random.randint(1, 6, size=data_size)
bathrooms = np.round(np.random.uniform(1, 4, size=data_size), 1)
house_age = np.random.randint(0, 100, size=data_size)

# Generate price
price = (square_footage * 150) + (bedrooms * 10000) + (bathrooms * 7000) - (house_age * 500) + np.random.normal(0, 20000, size=data_size)

# Create DataFrame
df = pd.DataFrame({
    "square_footage": square_footage,
    "bedrooms": bedrooms,
    "bathrooms": bathrooms,
    "house_age": house_age,
    "price": price
})

df.head()

In [ ]:
# Train-test split
X = df[["square_footage", "bedrooms", "bathrooms", "house_age"]]
y = df["price"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = LinearRegression()
model.fit(X_train, y_train)

# Save model
joblib.dump(model, "model.pkl")
print("✅ Model trained and saved as 'model.pkl'")

## 🚀 FastAPI Code

Copy the following code into a file named `main.py`.

In [ ]:
%%writefile main.py
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import joblib
import numpy as np

# Load the trained model
try:
    model = joblib.load("model.pkl")
except Exception as e:
    raise RuntimeError("Model loading failed")

app = FastAPI(title="House Price Predictor", description="Predict house prices using linear regression")

# Input schema
class PredictionInput(BaseModel):
    square_footage: float
    bedrooms: int
    bathrooms: float
    house_age: int

class PredictionOutput(BaseModel):
    prediction: float

# Health check
@app.get("/")
def health_check():
    return {"status": "healthy", "message": "House Price Prediction API is running"}

# Prediction endpoint
@app.post("/predict", response_model=PredictionOutput)
def predict(input_data: PredictionInput):
    try:
        features = np.array([[
            input_data.square_footage,
            input_data.bedrooms,
            input_data.bathrooms,
            input_data.house_age
        ]])
        prediction = model.predict(features)[0]
        return PredictionOutput(prediction=round(prediction, 2))
    except Exception as e:
        raise HTTPException(status_code=400, detail=str(e))

# Model info
@app.get("/model-info")
def model_info():
    return {
        "model_type": "LinearRegression",
        "problem_type": "Regression",
        "features": ["square_footage", "bedrooms", "bathrooms", "house_age"]
    }

## ▶️ Run FastAPI App (in terminal)

Once the model is trained and `main.py` is saved:

```bash
uvicorn main:app --reload
```

Visit: http://localhost:8000/docs to test the API.

## 📄 Example JSON Request for `/predict`
```json
{
  "square_footage": 2500,
  "bedrooms": 3,
  "bathrooms": 2.5,
  "house_age": 10
}
```